## Reactive Jump Pipeline Analysis
Date    :   2025/10/06  
Author  :   Salil Apte  
Version :   1.0  
Contact :   sal1l@pm.me  

This notebook demonstrates a full pipeline to compute **Reactive Strength Index (RSI)** from video-based toe position data.  
The workflow includes:

1. Loading the data
2. Detecting jumps (ground contact, toe-off, landing)
3. Computing RSI per foot (flight-time and position-peak methods)
4. Combining left and right foot RSI and computing asymmetry

In [ ]:
# Import Libraries
import pickle

# Import your existing classes
from event_detector import JumpEventDetector
from rsi_analyser import RSICalculator

### Load Data

Load the pickle file with the kinematics data and consider the three columns with left/right toe positions (in global frame) and timestamps.
- `toes_r_ty` (right toe vertical position in m)
- `toes_l_ty` (left toe vertical position in m)
- `time` (s)


In [ ]:
# Load data
data_dir = (
    r"E:\repos\mh-case\Data\subject0\OpenSimData\Kinematics\dvj_dl.pkl"
)
with open(data_dir, "rb") as f:
    kinematics = pickle.load(f)

# Get body kinematics and generalized coordinates
body_kinematics = kinematics["body_kinematics"]
generalized_coordinates = kinematics["generalized_coordinates"]

right_toe_y = body_kinematics["toes_r_ty"]
left_toe_y = body_kinematics["toes_l_ty"]
time = body_kinematics["time"]

# Preview data
body_kinematics.head()

## Detect Jump Events

We use the `JumpEventDetector` class to detect:
- **Ground contact**: start of stance phase after completing the drop jump
- **Toe-off**: end of stance phase before starting the reactive jump
- **Landing**: end of flight for reactive jump

This class also plots filtered positions and velocities with detected events for inspection.


In [ ]:
# Initialize detector
detector = JumpEventDetector(time, left_toe_y, right_toe_y)

# Plot positions and detected events
detector.plot()

# Inspect first few detected jumps
print("Left jumps (ground_contact, toe_off, landing):", detector.left_jumps[:1])
print("Right jumps (ground_contact, toe_off, landing):", detector.right_jumps[:1])

## Compute RSI Per Foot

We compute RSI using **two methods**:
1. **Flight-time method**: classical approach using time in air
2. **Position-peak method**: uses peak vertical displacement

The result is a DataFrame for with:
- Ground contact time (GCT)
- Peak height (flight & peak)
- RSI (flight & peak)

We combine left and right RSI per jump using the **median** and compute **asymmetry** as a percentage difference:

$$
\text{Asymmetry \%} = 2 \times \frac{|RSI_\text{Left} - RSI_\text{Right}|}{(RSI_\text{Left}+ RSI_\text{Right})} \times 100
$$

In [ ]:
rsi_calc = RSICalculator()

df_left = rsi_calc.compute_rsi_for_jumps(
    time, detector.left_filtered, detector.left_jumps, foot="Left"
)
df_right = rsi_calc.compute_rsi_for_jumps(
    time, detector.right_filtered, detector.right_jumps, foot="Right"
)
df_combined = rsi_calc.combine_feet(df_left, df_right)

# Inspect results
df_combined.head()